<a href="https://colab.research.google.com/github/bintangnabiil/Deep_Learning/blob/main/Deep_Learning_SVHN_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow_datasets as tfds
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt

# TensorFlow Dataset
(train_ds_tf, test_ds_tf), ds_info = tfds.load("svhn_cropped", split=["train", "test"], as_supervised=True, with_info=True)

# PyTorch Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_ds_torch = datasets.SVHN(root="./data", split="train", download=True, transform=transform)
test_ds_torch = datasets.SVHN(root="./data", split="test", download=True, transform=transform)

# DataLoader PyTorch
train_loader = DataLoader(train_ds_torch, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds_torch, batch_size=64, shuffle=False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.YO5HVJ_3.1.0/svhn_cropped-train.tfrecord*...:   0%…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.YO5HVJ_3.1.0/svhn_cropped-test.tfrecord*...:   0%|…

Generating extra examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.YO5HVJ_3.1.0/svhn_cropped-extra.tfrecord*...:   0%…

Dataset svhn_cropped downloaded and prepared to /root/tensorflow_datasets/svhn_cropped/3.1.0. Subsequent calls will reuse this data.


Model CNN

In [21]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

cnn_model = CNNModel()
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

Model MLP

In [22]:
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

mlp_model = MLPModel()
optimizer_mlp = optim.Adam(mlp_model.parameters(), lr=0.001)

Model CNN - Tensorflow

In [23]:
cnn_tf_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
cnn_tf_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model MLP - Tensorflow

In [24]:
mlp_tf_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
mlp_tf_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model Training

In [25]:
def train_model(model, loader, optimizer, criterion, epochs=5):
    for epoch in range(epochs):
        for images, labels in loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

train_model(cnn_model, train_loader, optimizer_cnn, criterion, epochs=5)
train_model(mlp_model, train_loader, optimizer_mlp, criterion, epochs=5)

cnn_tf_model.fit(train_ds_tf.batch(64), epochs=5)
mlp_tf_model.fit(train_ds_tf.batch(64), epochs=5)


Epoch 1/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 115s 98ms/step - accuracy: 0.5889 - loss: 2.7105
Epoch 2/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 141s 98ms/step - accuracy: 0.8510 - loss: 0.5120
Epoch 3/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 109s 95ms/step - accuracy: 0.8798 - loss: 0.4157
Epoch 4/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 109s 96ms/step - accuracy: 0.8931 - loss: 0.3602
Epoch 5/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 114s 100ms/step - accuracy: 0.9025 - loss: 0.3268
Epoch 1/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.1609 - loss: 59.1807
Epoch 2/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.1893 - loss: 2.2398
Epoch 3/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.1893 - loss: 2.2381
Epoch 4/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step - accuracy: 0.1893 - loss: 2.2381
Epoch 5/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.1893 - loss: 2.2381


Inisialisasi Model

In [26]:
cnn_model = CNNModel()
mlp_model = MLPModel()

Loss Function dan Optimizer

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=0.001)
optimizer_mlp = optim.Adam(mlp_model.parameters(), lr=0.001)

print("Model CNN dan MLP berhasil dibuat!")

Model CNN dan MLP berhasil dibuat!


Evaluasi Model CNN dan MLP


In [28]:
def evaluate_model(model, loader):
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(predicted.numpy())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="macro")
    rec = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    auc = roc_auc_score(y_true, np.eye(10)[y_pred], multi_class="ovr")

    print(f"Akurasi: {acc:.4f}, Presisi: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

print("Evaluasi CNN Model PyTorch:")
evaluate_model(cnn_model, test_loader)

print("\nEvaluasi MLP Model PyTorch:")
evaluate_model(mlp_model, test_loader)

print("\nEvaluasi CNN Model TensorFlow:")
cnn_tf_model.evaluate(test_ds_tf.batch(64))

print("\nEvaluasi MLP Model TensorFlow:")
mlp_tf_model.evaluate(test_ds_tf.batch(64))

Evaluasi CNN Model PyTorch:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Akurasi: 0.1946, Presisi: 0.0512, Recall: 0.1000, F1: 0.0380, AUC: 0.5000

Evaluasi MLP Model PyTorch:
Akurasi: 0.0727, Presisi: 0.1087, Recall: 0.0991, F1: 0.0490, AUC: 0.4994

Evaluasi CNN Model TensorFlow:
407/407 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step - accuracy: 0.8647 - loss: 0.5254

Evaluasi MLP Model TensorFlow:
407/407 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.2000 - loss: 2.2217


[2.2241573333740234, 0.19575907289981842]

ROC Curve dan AUC

In [30]:
from sklearn.preprocessing import label_binarize

# Fungsi menghitung AUC
def compute_auc(y_true, y_pred_proba):
    y_true_bin = label_binarize(y_true, classes=np.arange(10))  # Konversi label ke one-hot
    auc = roc_auc_score(y_true_bin, y_pred_proba, average='weighted', multi_class='ovr')
    print(f"AUC Score: {auc:.4f}")

# Fungsi evaluasi AUC untuk model TensorFlow
def evaluate_auc_tf(model, test_dataset):
    y_true = []
    y_pred_proba = []

    # Konversi dataset ke batch
    test_dataset = test_dataset.batch(64)  # Pastikan batch digunakan

    for images, labels in test_dataset:
        images = tf.convert_to_tensor(images)  # Konversi ke TensorFlow tensor
        predictions = model.predict(images, verbose=0)  # Nonaktifkan verbose
        y_pred_proba.extend(predictions.tolist())
        y_true.extend(labels.numpy().tolist())

    compute_auc(np.array(y_true), np.array(y_pred_proba))

# Evaluasi AUC untuk CNN TensorFlow
print("\nAUC Score untuk CNN TensorFlow:")
evaluate_auc_tf(cnn_tf_model, test_ds_tf)


AUC Score untuk CNN TensorFlow:
AUC Score: 0.9853


#Persamaan Matematika

##1. Fungsi Aktivasi ReLU
Fungsi aktivasi ReLU (Rectified Linear Unit) digunakan untuk memperkenalkan non-linearitas dalam model. Fungsi ini mendefinisikan bahwa jika input $x$ lebih besar dari nol, maka outputnya tetap $x$. Jika input negatif, maka outputnya nol. Persamaannya dinyatakan sebagai berikut:

$$f(x) = max(0,x)$$

Dalam kode, fungsi ReLU diterapkan setelah setiap lapisan konvolusi untuk membantu model mempelajari fitur yang lebih kompleks.<br><br>

##2. Operasi Konvolusi
Lapisan konvolusi digunakan untuk mengekstrak fitur dari gambar input dengan menggeser kernel (filter) di seluruh gambar dan menghitung hasil perkalian elemen. Operasi konvolusi dilakukan menggunakan persamaan:

$$y(i,j) = \sum\limits_{m=0}^{k-1} \sum\limits_{n=0}^{k-1} x(i+m,j+n) \times w(m,n)+b$$

Di mana:
- $x(i,j)$ = input gambar
- $w(m,n)$ = kernel konvolusi
- $b$ = bias
- $k$ = ukuran kernel (3x3 dalam kasus ini).<br><br>


##3. Batch Normalization
Teknik Batch Normalization digunakan untuk menstabilkan distribusi aktivasi di setiap batch pelatihan, sehingga model dapat berlatih lebih cepat dan lebih stabil. Normalisasi dilakukan dengan menghitung rata-rata $μ$ dan varians $σ^2$ dari batch input, kemudian menyesuaikan nilai input menggunakan persamaan berikut:

$$\hat{x} = \frac{x-𝜇}{\sqrt{σ^2+∈}}$$

Di mana:
- $𝜇$ = rata-rata mini-batch
- $σ^2$ = varians mini-batch
- $∈$ = nilai kecil untuk stabilitas numerik.<br><br>

##4. Max Pooling
Operasi Max Pooling digunakan untuk mereduksi dimensi fitur yang dihasilkan oleh lapisan konvolusi dengan hanya mengambil nilai maksimum dari setiap wilayah tertentu dalam fitur map. Persamaannya adalah:

$$y(i,j) = \underset{\text{(m,n)}}{\text{max}}I(i+m,j+n)$$

Hal ini membantu mengurangi jumlah parameter dalam model dan meningkatkan efisiensi komputasi sambil mempertahankan informasi fitur yang paling penting.<br><br>

##5. Fully Connected Layer (Linear Transformation)
Operasi Max Pooling digunakan untuk mereduksi dimensi fitur yang dihasilkan oleh lapisan konvolusi dengan hanya mengambil nilai maksimum dari setiap wilayah tertentu dalam fitur map. Persamaannya adalah:

$$y = W_x + b$$

Di mana:
- $W$ = matriks bobot
- $x$ = input
- $b$ = bias <br><br>

##6. Cross Entropy Loss
Fungsi Cross Entropy Loss digunakan sebagai metrik evaluasi dalam klasifikasi multi-kelas. Fungsi ini mengukur seberapa baik distribusi probabilitas prediksi dibandingkan dengan label yang sebenarnya. Persamaannya sebagai berikut:

$$L = - \sum\limits_{i-1}^C y_i log(\hat{y_i})$$

Di mana:
- $C$ = jumlah kelas
- $y_i$ = label sebenarnya
- $\hat{y_i}$ = probabilitas prediksi <br><br>

##7. Softmax (Untuk Probabilitas Kelas)
Fungsi Softmax digunakan untuk mengubah skor logit dari model menjadi probabilitas yang dapat dibandingkan antar kelas. Rumusnya adalah:

$$(\hat{y_i}) = \frac{e^{z_i}}{\sum_{j=1}^C e^{z_j}}$$

Di mana:
- $z_i$ = skor logit untuk kelas $i$ <br><br>

##8. AUC (Area Under Curve)
AUC (Area Under Curve) digunakan untuk mengevaluasi performa model klasifikasi dengan mengukur seberapa baik model membedakan antara kelas-kelas yang berbeda. AUC dihitung berdasarkan kurva ROC (Receiver Operating Characteristic), yang menggambarkan hubungan antara True Positive Rate (TPR) dan False Positive Rate (FPR). Persamaan untuk AUC adalah:

$$AUC = \int\limits_{0}^{1}TPR d FPR$$

Dengan:
- True Positive Rate (TPR) = $\frac{TP}{TP+FN}$
- False Positive Rate (FPR) = $\frac{FP}{FP+TN}$ <br><br>